In [25]:
import pandas as pd
import numpy as np
from glob import glob

def open_ephys_metadata(xml):
    import xml.etree.ElementTree as et
    import collections
    import pandas as pd
    def tryfloat (x):
        try: return float(x)
        except: return(x)
    tree = et.parse(xml)
    root = tree.getroot()
    StimConds = []
    for r in root.getchildren():
        StimCond = collections.OrderedDict()
        for e in r:
            StimCond[e.tag] = (tryfloat(e.text))
        StimConds.append(StimCond)
    columns = list(StimConds[0].keys())
    columns.remove('epoch')
    index = [s['epoch'] for s in StimConds]
    return pd.DataFrame(StimConds, index=index, columns=columns)

In [26]:
df = pd.read_csv('../Processing/data_list.csv')
expt_meta = glob(df.iloc[5]['dat_dir']+'ephys/*end.xml')[0]
expt_paradigm = open_ephys_metadata(expt_meta)

/groups/ahrens/home/weiz/miniconda3/envs/myenv/lib/python3.7/site-packages/ipykernel_launcher.py:15: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
  from ipykernel import kernelapp as app


In [27]:
probe_amp = expt_paradigm.loc['LG probe']['velocity']

In [28]:
probe_amp

1.3